<a href="https://colab.research.google.com/github/baraki-weldat/Cypher-Generation/blob/main/Final_ChatGPT_In_Context_Learning_with_Prompt_Engineering_GPT_3_5_Turbo_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Necessary Tools and Packages


In [1]:
%%capture
%pip install langchain
%pip install openai langchain-openai
%pip install langchainhub
%pip install neo4j
%pip install langchain-community langchain-core
%pip install openai==0.28
%pip install nltk rouge-score

# Import Implementation Packages


In [5]:
from google.colab import drive
import string
import pandas as pd
import numpy as np
from langchain_openai import ChatOpenAI
from langchain_community.graphs  import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain.chains.graph_qa.cypher_utils import CypherQueryCorrector, Schema
import openai
from neo4j import GraphDatabase
import nltk
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
from rouge_score import rouge_scorer

# Data Preparation and Cleaning

In [11]:
drive.mount("/content/drive", force_remount=True)
# Import the datasets
EvalCompanies = pd.read_excel("/content/drive/MyDrive/R and D from ABE/Raw Datasets/Companies Dataset.xlsx")
EvalMovies = pd.read_excel("/content/drive/MyDrive/R and D from ABE/Raw Datasets/Movies Dataset.xlsx")
EvalNetwork = pd.read_excel("/content/drive/MyDrive/R and D from ABE/Raw Datasets/Network Datasets.xlsx")

Mounted at /content/drive


In [12]:
# Drop unnecessary Columns
print(EvalCompanies.columns, EvalNetwork.columns, EvalMovies.columns)
EvalCompanies = EvalCompanies[['Natural Language Question', 'Cypher Query']]
EvalMovies = EvalMovies[['question', 'cypher']]
EvalNetwork = EvalNetwork[['Natural Language Question', 'Cypher Query']]
print(EvalCompanies.columns, EvalNetwork.columns, EvalMovies.columns)
# Rename Column names of the movies
EvalMovies= EvalMovies.rename(columns={'question': 'Natural Language Question', 'cypher': 'Cypher Query'})
print("The number of Evaluation datasets")
EvalCompanies.count(),EvalMovies.count(), EvalNetwork.count()
print("Total Evaluation datasets are:", len(EvalCompanies),len(EvalMovies),len(EvalNetwork))

# Drop entries with null values
EvalNetwork = EvalNetwork.dropna()
EvalMovies = EvalMovies.dropna()
EvalCompanies = EvalCompanies.dropna()
EvalNetwork.isnull().sum(), EvalMovies.isnull().sum(),EvalCompanies.isnull().sum()

# Rename Columns for Conformity
EvalMovies.columns =  ["Natural_Language_Question", "Cypher_Query"]
EvalCompanies.columns= ["Natural_Language_Question", "Cypher_Query"]
EvalNetwork.columns = ["Natural_Language_Question", "Cypher_Query"]

# Data Preprocessing
def Preprocess(text):
    PreProcessedText = text.lower().translate(str.maketrans('', '', string.punctuation))    # Convert to lowercase
    return PreProcessedText

# Preprocess the evaluation datasets
EvalMovies["Natural_Language_Question"]  = EvalMovies["Natural_Language_Question"].apply(Preprocess)
EvalCompanies["Natural_Language_Question"]  = EvalCompanies["Natural_Language_Question"].apply(Preprocess)
EvalNetwork["Natural_Language_Question"]  = EvalNetwork["Natural_Language_Question"].apply(Preprocess)


Index(['Natural Language Question', 'Cypher Query', 'Unnamed: 2'], dtype='object') Index(['Natural Language Question', 'Cypher Query'], dtype='object') Index(['question', 'cypher', 'validated_cypher', 'vote', 'has_answer',
       'database', 'database.1', 'Status'],
      dtype='object')
Index(['Natural Language Question', 'Cypher Query'], dtype='object') Index(['Natural Language Question', 'Cypher Query'], dtype='object') Index(['question', 'cypher'], dtype='object')
The number of Evaluation datasets
Total Evaluation datasets are: 139 265 69


# On Movies Dataset

In [ ]:
# @title
# Set up the OpenAI API key
openai.api_key = 'sk-proj-'


MOVIE_CYPHER_GENERATION_TEMPLATE = """
[INST]
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
    1. Use only the provided relationship types and properties in the schema.
    2. Do not use any other relationship types or properties that are not provided.
    3. If no data is returned, Generate the Cypher.
    4. Only respond to questions that require you to construct a Cypher statement.
    5. Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    6. Do not include any explanations or apologies in your responses.
    7. Do not include any text except the generated Cypher statement.
    8. Generate the Cypher, even if there are exceptions and errors in the query.

Schema: {schema}
Question: {question}
List all movies and persons acted in the movies.
[/INST]
Query:
QUERY: MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
RETURN m.title, p.name
"""

def generate_cypher(prompt, schema):
    formatted_prompt = MOVIE_CYPHER_GENERATION_TEMPLATE.format(schema=schema, question=prompt)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "user", "content": formatted_prompt}
        ]
    )

    # Extract the Cypher query from the response
    cypher_query = response['choices'][0]['message']['content'].strip()
    return cypher_query

def run_cypher_query(query):
    # Replace with your Neo4j connection details
    uri = "neo4j+s://demo.neo4jlabs.com"
    user = "movies"
    password = "movies"

    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        result = session.run(query)
        return result.values()

def evaluate_generated_query(description, expected_query, schema):
    generated_query = generate_cypher(description, schema)
    print(f"Natural_Language_Question: {description}")
    print(f"Generated Query: {generated_query}")
    print(f"Expected Query: {expected_query}")

    # Tokenize the queries
    generated_tokens = generated_query.split()
    expected_tokens = expected_query.split()

    Smooth = SmoothingFunction()
    # BLEU score
    bleu_score = sentence_bleu([expected_tokens], generated_tokens, smoothing_function=Smooth.method2 )

    # ROUGE score
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(expected_query, generated_query)

    return {
        "bleu": bleu_score,
        # "meteor": meteor,
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure }

if __name__ == "__main__":


    # Define the database schema
    schema = """
    Node properties are the following:
        Movie {title: STRING, votes: INTEGER, tagline: STRING, released: INTEGER}
        Person {born: INTEGER, name: STRING}
    Relationship properties are the following:
        ACTED_IN {roles: LIST}
        REVIEWED {summary: STRING, rating: INTEGER}
    The relationships are the following:
        (:Person)-[:ACTED_IN]->(:Movie)
        (:Person)-[:DIRECTED]->(:Movie)
        (:Person)-[:PRODUCED]->(:Movie)
        (:Person)-[:WROTE]->(:Movie)
        (:Person)-[:FOLLOWS]->(:Person)
        (:Person)-[:REVIEWED]->(:Movie)
    """

    # Evaluate the generated Cypher queries
    results = []
    total = len(EvalMovies)

    for index, row in EvalMovies.iterrows():
        NLQuestion = row['Natural_Language_Question']
        expected_query = row['Cypher_Query']
        metrics = evaluate_generated_query(NLQuestion, expected_query, schema)
        results.append(metrics)
    # Calculate average scores
    avg_bleu = sum(result['bleu'] for result in results) / total
    # avg_meteor = sum(result['meteor'] for result in results) / total
    avg_rouge1 = sum(result['rouge1'] for result in results) / total
    avg_rougeL = sum(result['rougeL'] for result in results) / total

    print(f"Average BLEU Score: {avg_bleu:.2f}")
    # print(f"Average METEOR Score: {avg_meteor:.2f}")
    print(f"Average ROUGE-1 Score: {avg_rouge1:.2f}")
    print(f"Average ROUGE-L Score: {avg_rougeL:.2f}")


# Companies Dataset


In [ ]:
# Set up the OpenAI API key
openai.api_key = 'sk-proj-'

COM_CYPHER_GENERATION_TEMPLATE = """
[INST]
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about Companies and provide information of the persons involved in.
Convert the user's question based on the schema.

Instructions:
    1. Use only the provided relationship types and properties in the schema.
    2. Do not use any other relationship types or properties that are not provided.
    3. Even If no data is returned, Generate the Cypher.
    4. Only respond to questions that require you to construct a Cypher statement.
    5. Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    6. Do not include any explanations or apologies in your responses.
    7. Do not include any text except the generated Cypher statement.
    8. Generate the Cypher, even if there are exceptions and errors in the query.

Schema: {schema}
Question: {question}
List all organizations headquartered in New York.
[/INST]
Query:
MATCH (org:Organization)-[:IN_CITY]->(:City {{name: "New York City"}})
RETURN org.name as ListOfOrganizations
"""

def generate_cypher(prompt, schema):
    formatted_prompt = COM_CYPHER_GENERATION_TEMPLATE.format(schema=schema, question=prompt)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": formatted_prompt}
        ]
    )

    # Extract the Cypher query from the response
    cypher_query = response['choices'][0]['message']['content'].strip()
    return cypher_query

def run_cypher_query(query):
    # Replace with your Neo4j connection details
    uri = "neo4j+s://demo.neo4jlabs.com"
    user = "companies"
    password = "companies"

    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        result = session.run(query)
        return result.values()

def evaluate_generated_query(question, expected_query, schema):
    generated_query = generate_cypher(question, schema)
    print(f"Natural_Language_Question: {question}")
    print(f"Generated Query: {generated_query}")
    print(f"Expected Query: {expected_query}")

    # Tokenize the queries
    generated_tokens = generated_query.split()
    expected_tokens = expected_query.split()

    Smooth = SmoothingFunction() # method0 .... method7 (0.14, 0.20, 0.30, 0.29,0.21, 0.21, err, 0.27 )
    # BLEU score
    bleu_score = sentence_bleu([expected_tokens], generated_tokens, smoothing_function=Smooth.method2 )

    # ROUGE score
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(expected_query, generated_query)

    return {
        "bleu": bleu_score,
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure
    }

if __name__ == "__main__":

    # Define the schema
    schema = """
    Node properties are the following:
    _Bloom_Scene_ {{ranges: STRING, gds: STRING, createdAt: INTEGER, lastModified: INTEGER, style: STRING, createdBy: STRING, visualisation: STRING, version: STRING, roles: LIST, numOfNodes: INTEGER, numOfRels: INTEGER, id: STRING, relationships: STRING, name: STRING, nodes: STRING}},
    _Bloom_Perspective_ {{data: STRING, version: STRING, roles: LIST, name: STRING, id: STRING}},
    Person {{name: STRING, id: STRING, summary: STRING}},
    Organization {{name: STRING, nbrEmployees: INTEGER, isDissolved: BOOLEAN, id: STRING, motto: STRING, summary: STRING, isPublic: BOOLEAN, revenue: FLOAT}},
    IndustryCategory {{name: STRING, id: STRING}},
    City {{id: STRING, summary: STRING, name: STRING}},
    Country {{name: STRING, id: STRING, summary: STRING}},
    Article {{id: STRING, sentiment: FLOAT, author: STRING, siteName: STRING, summary: STRING, date: DATE_TIME, title: STRING}},
    Chunk {{text: STRING, embedding: LIST, embedding_google: LIST}},
    Fewshot {{Question: STRING, Cypher: STRING, id: INTEGER, embedding: LIST}}

    Relationship properties are the following:

    The relationships are the following:
    (:_Bloom_Perspective_)-[:_Bloom_HAS_SCENE_]->(:_Bloom_Scene_),
    (:Person)-[:HAS_PARENT]->(:Person),
    (:Person)-[:HAS_CHILD]->(:Person),
    (:Organization)-[:HAS_CEO]->(:Person),
    (:Organization)-[:IN_CITY]->(:City),
    (:Organization)-[:HAS_CATEGORY]->(:IndustryCategory),
    (:Organization)-[:HAS_SUBSIDIARY]->(:Organization),
    (:Organization)-[:HAS_SUPPLIER]->(:Organization),
    (:Organization)-[:HAS_BOARD_MEMBER]->(:Person),
    (:Organization)-[:HAS_INVESTOR]->(:Organization),
    (:Organization)-[:HAS_INVESTOR]->(:Person),
    (:Organization)-[:HAS_COMPETITOR]->(:Organization),
    (:City)-[:IN_COUNTRY]->(:Country),
    (:Article)-[:HAS_CHUNK]->(:Chunk),
    (:Article)-[:MENTIONS]->(:Organization)
    """

    # Evaluate the generated Cypher queries
    results = []
    total = len(EvalCompanies)

    for index, row in EvalCompanies.iterrows():
        ComNLQuestion = row['Natural_Language_Question']
        Com_expected_query = row['Cypher_Query']
        metrics = evaluate_generated_query(ComNLQuestion, Com_expected_query, schema)
        results.append(metrics)

    # Calculate average scores
    avg_bleu = sum(result['bleu'] for result in results) / total
    avg_rouge1 = sum(result['rouge1'] for result in results) / total
    avg_rougeL = sum(result['rougeL'] for result in results) / total

    print(f"Average BLEU Score: {avg_bleu:.2f}")
    print(f"Average ROUGE-1 Score: {avg_rouge1:.2f}")
    print(f"Average ROUGE-L Score: {avg_rougeL:.2f}")

# On Netowrk Dataset

In [ ]:
# Set up the OpenAI API key
openai.api_key = 'sk-proj---'

NET_CYPHER_GENERATION_TEMPLATE = """
[INST]
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about Network infrustracture components information.
Convert the user's question based on the schema.

Instructions:
    1. Use only the provided relationship types and properties in the schema.
    2. Do not use any other relationship types or properties that are not provided.
    3. Even If no data is returned, Generate the Cypher.
    4. Only respond to questions that require you to construct a Cypher statement.
    5. Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    6. Do not include any explanations or apologies in your responses.
    7. Do not include any text except the generated Cypher statement.
    8. Generate the Cypher, even if there are exceptions and errors in the query.

Schema: {schema}
Question: {question}
Retrieve all Routers connected to an interface
[/INST]
Query:
MATCH (r:Router)-[:ROUTES]->(i:Interface)
RETURN r.name
"""

def generate_cypher(prompt, schema):
    formatted_prompt = NET_CYPHER_GENERATION_TEMPLATE.format(schema=schema, question=prompt)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": formatted_prompt}
        ]
    )

    # Extract the Cypher query from the response
    cypher_query = response['choices'][0]['message']['content'].strip()
    return cypher_query

def run_cypher_query(query):
    # Replace with your Neo4j connection details
    uri = "neo4j+s://demo.neo4jlabs.com"
    user = "network"
    password = "network"

    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        result = session.run(query)
        return result.values()

def evaluate_generated_query(question, expected_query, schema):
    generated_query = generate_cypher(question, schema)
    print(f"Natural_Language_Question: {question}")
    print(f"Generated Query: {generated_query}")
    print(f"Expected Query: {expected_query}")

    # Tokenize the queries
    generated_tokens = generated_query.split()
    expected_tokens = expected_query.split()

    Smooth = SmoothingFunction() # method0 .... method7 (0.14, 0.20, 0.30, 0.29,0.21, 0.21, err, 0.27 )
    # BLEU score
    bleu_score = sentence_bleu([expected_tokens], generated_tokens, smoothing_function=Smooth.method2 )

    # ROUGE score
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(expected_query, generated_query)

    return {
        "bleu": bleu_score,
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure
    }

if __name__ == "__main__":

    # Define the schema as a string (this should match the actual schema of your Neo4j database)
    schema = """
    Node properties are the following:
      DataCenter {name: STRING, location: STRING},
      Router {name: STRING, zone: INTEGER},
      Egress {name: STRING}, Interface {ip: STRING},
      Network {ip: STRING, size: INTEGER, zone: INTEGER},
      Zone {ip: STRING, size: INTEGER, zone: INTEGER},
      Rack {name: STRING, zone: INTEGER, rack: INTEGER},
      Switch {ip: STRING, rack: INTEGER},
      Type {id: INTEGER, type: STRING, ram: INTEGER, name: STRING, disk: INTEGER, cpu: INTEGER},
      Machine {name: STRING}, Software {name: STRING, versions: LIST, ports: LIST, dependencies: LIST},
      OS {name: STRING, startTime: INTEGER}, Service {name: STRING, startTime: INTEGER, pid: INTEGER},
      Application {name: STRING, startTime: INTEGER, pid: INTEGER}, Version {name: STRING},
      Process {name: STRING, startTime: INTEGER, pid: INTEGER}, Port {port: INTEGER}
    Relationship properties are the following:

    The relationships are the following:
      (:DataCenter)-[:CONTAINS]->(:Rack), (:DataCenter)-[:CONTAINS]->(:Router),
      (:DataCenter)-[:CONTAINS]->(:Egress), (:Router)-[:ROUTES]->(:Interface), (:Egress)-[:ROUTES]->(:Interface),
      (:Interface)-[:EXPOSES]->(:Port), (:Interface)-[:CONNECTS]->(:Interface), (:Network)-[:ROUTES]->(:Interface),
      (:Zone)-[:ROUTES]->(:Interface), (:Rack)-[:HOLDS]->(:Machine), (:Rack)-[:HOLDS]->(:Switch), (:Switch)-[:ROUTES]->(:Interface),
      (:Machine)-[:RUNS]->(:Application), (:Machine)-[:RUNS]->(:Process), (:Machine)-[:RUNS]->(:OS), (:Machine)-[:RUNS]->(:Service),
      (:Machine)-[:ROUTES]->(:Interface), (:Machine)-[:TYPE]->(:Type), (:Software)-[:VERSION]->(:Version),
      (:Software)-[:DEPENDS_ON]->(:Version), (:Software)-[:DEPENDS_ON]->(:Software), (:Software)-[:DEPENDS_ON]->(:Application),
      (:Software)-[:DEPENDS_ON]->(:Service), (:OS)-[:INSTANCE]->(:Version), (:Service)-[:INSTANCE]->(:Software),
      (:Service)-[:INSTANCE]->(:Version), (:Service)-[:INSTANCE]->(:Application), (:Service)-[:INSTANCE]->(:Service),
      (:Service)-[:LISTENS]->(:Port), (:Application)-[:INSTANCE]->(:Software), (:Application)-[:INSTANCE]->(:Application),
      (:Application)-[:LISTENS]->(:Port), (:Application)-[:DEPENDS_ON]->(:Service), (:Application)-[:DEPENDS_ON]->(:Process),
      (:Version)-[:PREVIOUS]->(:Version), (:Process)-[:LISTENS]->(:Port), (:Process)-[:INSTANCE]->(:Software),
      (:Process)-[:INSTANCE]->(:Application), (:Process)-[:INSTANCE]->(:Version), (:Process)-[:INSTANCE]->(:Service),
      (:Process)-[:DEPENDS_ON]->(:Service), (:Process)-[:DEPENDS_ON]->(:Process)
    """

    # Evaluate the generated Cypher queries
    results = []
    total = len(EvalNetwork)

    for index, row in EvalNetwork.iterrows():
        NetNLQuestion = row['Natural_Language_Question']
        Net_expected_query = row['Cypher_Query']
        metrics = evaluate_generated_query(NetNLQuestion, Net_expected_query, schema)
        results.append(metrics)

    # Calculate average scores
    avg_bleu = sum(result['bleu'] for result in results) / total
    avg_rouge1 = sum(result['rouge1'] for result in results) / total
    avg_rougeL = sum(result['rougeL'] for result in results) / total

    print(f"Average BLEU Score: {avg_bleu:.2f}")
    print(f"Average ROUGE-1 Score: {avg_rouge1:.2f}")
    print(f"Average ROUGE-L Score: {avg_rougeL:.2f}")